# Hallucination as Signal Decay: Interactive Walkthrough

Welcome! This notebook accompanies our paper **"When Models Know but Don’t Say"**. 

We're going to explore why LLMs often fail to produce facts they actually "know" (based on their internal states) and demonstrate how to recover that knowledge using **Logit Mixing**.

### What we'll do:
1.  **Reproduce the "Knowledge Gap"**: See the massive difference between what the model *knows* (Pairwise) and what it *says* (Generation).
2.  **Trace the Signal**: Use the **Logit Lens** to watch the truth decay in real-time across layers.
3.  **Fix it**: Apply our lightweight intervention to rescue the correct answer.

> **Note**: This notebook uses `meta-llama/Llama-3.1-8B` by default. You'll need a GPU with ~24GB VRAM (or use 4-bit quantization).


In [ ]:
import sys
import os
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformer_lens import HookedTransformer

# Add src to path so we can import our modules
sys.path.append(os.path.abspath("src"))

# Set up plotting style
sns.set_context("notebook", font_scale=1.2)
sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'sans-serif'

print("Environment set up. Ready to load models.")

## 1. Load the Model

We use `TransformerLens` for easy access to internal activations. 
*Tip: If you're low on memory, uncomment the `load_in_4bit` logic (though exact values might shift slightly).*


In [ ]:
# Config
MODEL_NAME = "meta-llama/Llama-3.1-8B"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading {MODEL_NAME} on {DEVICE}...")

# Load model (Standard FP16 for reproduction)
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    device=DEVICE,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False
)

print("Model loaded!")

## 2. The Knowledge Gap

Let's look at a few examples from the **CounterFact** dataset. We'll check two things:
1.  **Pairwise Accuracy**: Does the model assign higher probability to the *True* object than the *False* one? (i.e., Does it know?)
2.  **Generation**: If we just let it generate, does it output the True object?


In [ ]:
from datasets import load_dataset
from src.utils.eval_utils import get_answer_logits

# Load a tiny subset for demo purposes
dataset = load_dataset("NeelNanda/counterfact-tracing", split="train[:10]")

print(f"Checking {len(dataset)} examples...")

results = []
for item in dataset:
    prompt = item['prompt']
    target_true = item['target_true'].strip()
    target_false = item['target_false'].strip()
    
    # Run model
    logits = model(prompt, prepend_bos=True)
    last_token_logits = logits[0, -1, :]
    
    # Check Gen
    pred_token = model.to_string(torch.argmax(last_token_logits))
    is_correct_gen = (pred_token.strip().lower() == target_true.lower())
    
    # Check Pairwise
    true_logit = last_token_logits[model.to_single_token(f" {target_true}")]
    false_logit = last_token_logits[model.to_single_token(f" {target_false}")]
    is_correct_pair = (true_logit > false_logit).item()
    
    results.append({
        "Prompt": prompt,
        "True": target_true,
        "Pred": pred_token,
        "Pairwise": "✅" if is_correct_pair else "❌",
        "Gen": "✅" if is_correct_gen else "❌"
    })

df_gap = pd.DataFrame(results)
display(df_gap)

gap = df_gap[df_gap['Pairwise'] == '✅'][df_gap['Gen'] == '❌']
print(f"\nFound {len(gap)} examples where the model KNOWS but FAILS to generate.")


## 3. Investigating the "Signal Decay"

Take one of those failed examples (where Pairwise is ✅ but Gen is ❌). What happened inside the model? 

We'll use a **Logit Lens** trace. We decode the residual stream at every layer into the vocabulary.
*   **Hypothesis**: The model *had* the correct token in the middle layers, but lost it.


In [ ]:
# Pick the first "Overwritten" error
if len(gap) > 0:
    example = gap.iloc[0]
else:
    # Fallback if 10 examples didn't catch one (unlikely, usually ~40% rate)
    example = df_gap.iloc[0] 

prompt = example['Prompt']
target = example['True']

print(f"Analyzing Prompt: '{prompt}'")
print(f"Target: '{target}'")

# Run cache
logits, cache = model.run_with_cache(prompt)
target_id = model.to_single_token(f" {target}")

layer_probs = []

for layer in range(model.cfg.n_layers):
    # Decode residual stream at this layer
    resid = cache[f"blocks.{layer}.hook_resid_post"][0, -1, :]
    normalized = model.ln_final(resid)
    layer_logits = model.unembed(normalized)
    
    # Get prob of target
    probs = torch.softmax(layer_logits, dim=-1)
    target_prob = probs[target_id].item()
    layer_probs.append(target_prob)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(layer_probs, marker='o', linewidth=2.5, color='#d62728')
plt.title(f"Trace of Token: '{target}'")
plt.xlabel("Layer")
plt.ylabel("Probability")
plt.axhline(0, color='gray', linestyle='--')
plt.grid(True, alpha=0.3)
plt.show()

print("Look at that trajectory! Often you see a peak around Layer 20, followed by a collapse.")

## 4. The Fix: Logit Mixing

Now that we know the "truth" is strong in the mid-layers, let's just helpful nudge the final output.

We define a mixing intervention:
$$ L_{final} = (1 - \alpha) \cdot L_{final} + \alpha \cdot L_{mid} $$

Let's try $\alpha=0.2$ (mixing in 20% of Layer 20).


In [ ]:
alpha = 0.2
mid_layer = 20

# Get mid logits (re-using cache)
resid_mid = cache[f"blocks.{mid_layer}.hook_resid_post"][0, -1, :]
norm_mid = model.ln_final(resid_mid)
logits_mid = model.unembed(norm_mid)

# Get final logits
logits_final = logits[0, -1, :]

# Mix
logits_mixed = (1 - alpha) * logits_final + (alpha) * logits_mid

# Check result
new_pred_id = torch.argmax(logits_mixed)
new_pred = model.to_string(new_pred_id)

print(f"Original Gen: '{example['Pred']}'")
print(f"Intervention (alpha={alpha}): '{new_pred}'")

if new_pred.strip().lower() == target.lower():
    print("🚀 SUCCESS: Knowledge recovered!")
else:
    print("No change. Maybe try a higher alpha?")

## Wrap Up

This was a simplified demo of the core finding: **Signal Decay**.

The full analysis in the paper (and the `src/` folder) scales this up to thousands of examples, trains a dedicated probe to detect *when* to intervene (so we don't break correct answers), and evaluates OOD robustness.

Feel free to play around with the `alpha` or try different prompts!
